## Import required modules and Initialize SparkSession

In [1]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()

'C:\\dev-tools\\spark-3.0.0-bin-hadoop2.7'

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Aggregations").getOrCreate()

## Create Dataframe with sample data

In [5]:
schema = ['Year','Course','Fee']
data = [(2017,'Spark',15000),(2017,'Hadoop',10000),(2017,'Scala',10000),(2017,'HBase',10000),(2017,'Java',10000),
       (2016,'Spark',20000),(2016,'Hadoop',15000),(2016,'Scala',10000),(2016,'HBase',10000),(2016,'Java',10000),
       (2015,'Spark',20000),(2015,'Hadoop',20000),(2015,'Scala',10000),(2015,'HBase',10000),(2015,'Java',10000),
       (2014,'Cassandra',20000),(2014,'Hadoop',20000),(2014,'Android',15000),(2014,'HBase',15000),(2014,'Java',10000)]
print(schema)
print(data)

['Year', 'Course', 'Fee']
[(2017, 'Spark', 15000), (2017, 'Hadoop', 10000), (2017, 'Scala', 10000), (2017, 'HBase', 10000), (2017, 'Java', 10000), (2016, 'Spark', 20000), (2016, 'Hadoop', 15000), (2016, 'Scala', 10000), (2016, 'HBase', 10000), (2016, 'Java', 10000), (2015, 'Spark', 20000), (2015, 'Hadoop', 20000), (2015, 'Scala', 10000), (2015, 'HBase', 10000), (2015, 'Java', 10000), (2014, 'Cassandra', 20000), (2014, 'Hadoop', 20000), (2014, 'Android', 15000), (2014, 'HBase', 15000), (2014, 'Java', 10000)]


In [6]:
courses_df = spark.createDataFrame(data,schema)

In [7]:
courses_df.printSchema()

root
 |-- Year: long (nullable = true)
 |-- Course: string (nullable = true)
 |-- Fee: long (nullable = true)



In [8]:
courses_df.show()

+----+---------+-----+
|Year|   Course|  Fee|
+----+---------+-----+
|2017|    Spark|15000|
|2017|   Hadoop|10000|
|2017|    Scala|10000|
|2017|    HBase|10000|
|2017|     Java|10000|
|2016|    Spark|20000|
|2016|   Hadoop|15000|
|2016|    Scala|10000|
|2016|    HBase|10000|
|2016|     Java|10000|
|2015|    Spark|20000|
|2015|   Hadoop|20000|
|2015|    Scala|10000|
|2015|    HBase|10000|
|2015|     Java|10000|
|2014|Cassandra|20000|
|2014|   Hadoop|20000|
|2014|  Android|15000|
|2014|    HBase|15000|
|2014|     Java|10000|
+----+---------+-----+



## Perform aggregation on the sample data

In [13]:
group_df = courses_df.groupBy('Year')

In [14]:
group_df.show()

AttributeError: 'GroupedData' object has no attribute 'show'

In [15]:
type(group_df)

pyspark.sql.group.GroupedData

In [16]:
group_df.count().show()

+----+-----+
|Year|count|
+----+-----+
|2014|    5|
|2016|    5|
|2017|    5|
|2015|    5|
+----+-----+



In [17]:
# Find earning per Year
group_df.sum('Fee').show()

+----+--------+
|Year|sum(Fee)|
+----+--------+
|2014|   80000|
|2016|   65000|
|2017|   55000|
|2015|   70000|
+----+--------+



In [18]:
# Find average earning per Year
group_df.avg('Fee').show()

+----+--------+
|Year|avg(Fee)|
+----+--------+
|2014| 16000.0|
|2016| 13000.0|
|2017| 11000.0|
|2015| 14000.0|
+----+--------+



In [19]:
from pyspark.sql import functions as F
group_df.agg(F.min('Fee').alias('Min'), 
             F.max('Fee').alias('Max'),
             F.sum('Fee').alias('Earnings'),
             F.avg('Fee').alias('Average Earnings'),
             F.count('*').alias('Num of Courses')
            ).show()

+----+-----+-----+--------+----------------+--------------+
|Year|  Min|  Max|Earnings|Average Earnings|Num of Courses|
+----+-----+-----+--------+----------------+--------------+
|2014|10000|20000|   80000|         16000.0|             5|
|2016|10000|20000|   65000|         13000.0|             5|
|2017|10000|15000|   55000|         11000.0|             5|
|2015|10000|20000|   70000|         14000.0|             5|
+----+-----+-----+--------+----------------+--------------+



In [20]:
# Compute the sum of earnings for each year by course with each course as a separate column
group_df.pivot('Course',['Java','Hadoop','HBase','Scala','Spark','Casandra','Android']).sum('Fee').show()

+----+-----+------+-----+-----+-----+--------+-------+
|Year| Java|Hadoop|HBase|Scala|Spark|Casandra|Android|
+----+-----+------+-----+-----+-----+--------+-------+
|2014|10000| 20000|15000| null| null|    null|  15000|
|2016|10000| 15000|10000|10000|20000|    null|   null|
|2017|10000| 10000|10000|10000|15000|    null|   null|
|2015|10000| 20000|10000|10000|20000|    null|   null|
+----+-----+------+-----+-----+-----+--------+-------+



In [27]:
# Get count of distinct items in a group.
courses_df.select(F.approx_count_distinct("Course")).show()

+-----------------------------+
|approx_count_distinct(Course)|
+-----------------------------+
|                            7|
+-----------------------------+



In [33]:
# Get all values from an input column (with duplicates)
courses_df.select(F.collect_list("Course")).show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+
|collect_list(Course)                                                                                                                              |
+--------------------------------------------------------------------------------------------------------------------------------------------------+
|[Spark, Hadoop, Scala, HBase, Java, Spark, Hadoop, Scala, HBase, Java, Spark, Hadoop, Scala, HBase, Java, Cassandra, Hadoop, Android, HBase, Java]|
+--------------------------------------------------------------------------------------------------------------------------------------------------+



In [34]:
# Get all values from an input column (without duplicates)
courses_df.select(F.collect_set("Course")).show(truncate=False)

+-------------------------------------------------------+
|collect_set(Course)                                    |
+-------------------------------------------------------+
|[Scala, Spark, Java, Cassandra, HBase, Android, Hadoop]|
+-------------------------------------------------------+



In [36]:
courses_df.select(F.count("Course")).show()

+-------------+
|count(Course)|
+-------------+
|           20|
+-------------+

